# Toxicity Classification Homework 2

This notebook demonstrates the basic requirements of the civility in communication assignment. We evaluate a rule-based classifier using PerspectiveAPI scores and train a simple logistic regression classifier using TF‑IDF features. We compute performance metrics and false positive rates (FPR) across demographic groups.


In [ ]:

import pandas as pd
from toxicity_model import read_data, train_classifier, evaluate_model, compute_fpr, rule_based_perspective, predict_and_save

data_dir = 'civility_data'
train_df = read_data(f"{data_dir}/train.tsv")
dev_df = read_data(f"{data_dir}/dev.tsv")
demo_df = read_data(f"{data_dir}/mini_demographic_dev.tsv")
test_df = read_data(f"{data_dir}/test.tsv")

train_df.head(), dev_df.head(), demo_df.head()


In [ ]:

# Rule-based classifier using PerspectiveAPI scores > 0.8
rule_preds = rule_based_perspective(dev_df)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

acc = accuracy_score(dev_df['label'], rule_preds)
prec, rec, f1, _ = precision_recall_fscore_support(dev_df['label'], rule_preds, labels=['NOT','OFF'], average=None)

print('Rule-based classifier results:')
print(f'Accuracy: {acc:.3f}')
print(f'Precision NOT: {prec[0]:.3f}, Recall NOT: {rec[0]:.3f}, F1 NOT: {f1[0]:.3f}')
print(f'Precision OFF: {prec[1]:.3f}, Recall OFF: {rec[1]:.3f}, F1 OFF: {f1[1]:.3f}')

# FPR for PerspectiveAPI on demographic dev set
rule_demo_preds = rule_based_perspective(demo_df)
rule_fpr = compute_fpr(rule_demo_preds, demo_df['demographic'])
print('
Rule-based False Positive Rate per demographic group:')
for grp, rate in rule_fpr.items():
    print(f'{grp}: {rate:.3f}')


In [ ]:

# Train logistic regression classifier
clf = train_classifier(train_df)
results = evaluate_model(clf, dev_df)
print('Logistic Regression evaluation:')
for k,v in results.items():
    print(f'{k}: {v:.3f}')


In [ ]:

log_preds_demo = clf.predict(demo_df['text'])
log_fpr = compute_fpr(log_preds_demo, demo_df['demographic'])
print('Logistic Regression False Positive Rate per demographic group:')
for grp, rate in log_fpr.items():
    print(f'{grp}: {rate:.3f}')


In [ ]:

# Predict on test and save
output_file = 'FirstName_LastName_test.tsv'
predict_and_save(clf, test_df, output_file)
print(f'Test predictions saved to {output_file}')
